## Feature Enginnering

In [0]:
import pandas as pd

train_data = pd.read_csv('./train.csv')
test_data = pd.read_csv('./test.csv')
data = [train_data, test_data]

In [15]:
train_data.isnull().sum().to_frame()

,0
PassengerId,0
Survived,0
Pclass,0
Name,0
Sex,0
Age,177
SibSp,0
Parch,0
Ticket,0
Fare,0


In [16]:
test_data.isnull().sum().to_frame()

,0
PassengerId,0
Pclass,0
Name,0
Sex,0
Age,86
SibSp,0
Parch,0
Ticket,0
Fare,1
Cabin,327


In [0]:
for d in data:
  d['Initial'] = d['Name'].str.extract('([A-Za-z]+)\.')

In [12]:
pd.crosstab(train_data.Sex, train_data.Initial)

Initial,Capt,Col,Countess,Don,Dr,Jonkheer,Lady,Major,Master,Miss,Mlle,Mme,Mr,Mrs,Ms,Rev,Sir
Sex,,,,,,,,,,,,,,,,,
female,0,0,1,0,1,0,1,0,0,182,2,1,0,125,1,0,0
male,1,2,0,1,6,1,0,2,40,0,0,0,517,0,0,6,1


In [13]:
pd.crosstab(test_data.Sex, train_data.Initial)

Initial,Don,Dr,Master,Miss,Mme,Mr,Mrs,Rev
Sex,,,,,,,,
female,0,0,9,37,0,83,21,2
male,1,3,14,64,1,145,37,1


In [0]:
for d in data:
  d.Initial.replace(
    ['Mlle', 'Mme', 'Ms', 'Dr', 'Major', 'Capt', 'Sir', 'Don', 'Dona', 'Lady', 'Countess', 'Jonkheer', 'Rev', 'Col'],
    ['Miss', 'Miss', 'Miss', 'Mr', 'Mr', 'Mr', 'Mr', 'Mr', 'Mrs', 'Mrs', 'Mrs', 'Other', 'Other', 'Other'],
    inplace=True
  )

In [0]:
for d in data:
  d.loc[(d.Initial=='Master')&(d.Age.isnull()), 'Age'] = 5
  d.loc[(d.Initial=='Miss')&(d.Age.isnull()), 'Age'] = 22
  d.loc[(d.Initial=='Mr')&(d.Age.isnull()), 'Age'] = 33
  d.loc[(d.Initial=='Mrs')&(d.Age.isnull()), 'Age'] = 36
  d.loc[(d.Initial=='Other')&(d.Age.isnull()), 'Age'] = 46

In [20]:
print(test_data.Age.isnull().any(), train_data.Age.isnull().any())

False False


In [22]:
for d in data:
  print(d.Initial.value_counts())

Mr        529
Miss      186
Mrs       127
Master     40
Other       9
Name: Initial, dtype: int64
Mr        241
Miss       79
Mrs        73
Master     21
Other       4
Name: Initial, dtype: int64


In [26]:
train_data.Embarked.value_counts().idxmax()

'S'

In [27]:
train_data.Embarked.fillna('S', inplace=True)
train_data.Embarked.isnull().any()

False

In [28]:
print(test_data[test_data.Fare.isnull()])

     PassengerId  Pclass                Name   Sex  ...  Fare  Cabin  Embarked Initial
152         1044       3  Storey, Mr. Thomas  male  ...   NaN    NaN         S      Mr

[1 rows x 12 columns]


In [32]:
test_data[test_data.Pclass==3].Fare.mean()

12.459677880184334

In [34]:
test_data.Fare.fillna(12.5, inplace=True)
test_data.Fare.isnull().any()

False

In [0]:
for d in data:
  d['Age_band'] = 0
  d.loc[d.Age <= 16, 'Age_band'] = 0
  d.loc[(d.Age > 16) & (d.Age <= 32), 'Age_band'] = 1
  d.loc[(d.Age > 32) & (d.Age <= 48), 'Age_band'] = 2
  d.loc[(d.Age > 48) & (d.Age <= 64), 'Age_band'] = 3
  d.loc[d.Age > 64, 'Age_band'] = 4

In [0]:
for d in data:
  d['Family_size'] = d['SibSp'] + d['Parch']
  d['Alone'] = 0
  d.loc[d.Family_size==0, 'Alone'] = 1

In [0]:
for d in data:
  d['Fare_range'] = pd.qcut(d.Fare, 4)

In [0]:
for d in data:
  d['Fare_cat'] = 0
  d.loc[d.Fare <= 7.91, 'Fare_cat'] = 0
  d.loc[(d.Fare > 7.91) & (d.Fare <= 14.454), 'Fare_cat'] = 1
  d.loc[(d.Fare > 14.454) & (d.Fare <= 31.0), 'Fare_cat'] = 2
  d.loc[d.Fare > 31.0, 'Fare_cat'] = 3

In [0]:
for d in data:
  d.Sex.replace(['male', 'female'], [0, 1], inplace=True)
  d.Embarked.replace(['S', 'C', 'Q'], [0, 1, 2], inplace=True)
  d.Initial.replace(['Mr', 'Mrs', 'Miss', 'Master', 'Other'], [0, 1, 2, 3, 4], inplace=True)

In [0]:
for d in data:
  d.drop(['Name', 'Age', 'Ticket', 'Cabin', 'Fare', 'Fare_range', 'PassengerId'], axis=1, inplace=True)

In [43]:
train_data.isnull().sum().to_frame()

,0
Survived,0
Pclass,0
Sex,0
SibSp,0
Parch,0
Embarked,0
Initial,0
Age_band,0
Family_size,0
Alone,0


In [44]:
test_data.isnull().sum().to_frame()

,0
Pclass,0
Sex,0
SibSp,0
Parch,0
Embarked,0
Initial,0
Age_band,0
Family_size,0
Alone,0
Fare_cat,0


## Prediction

In [45]:
from sklearn.ensemble import AdaBoostClassifier

X = train_data[train_data.columns[1:]]
y = train_data.Survived

model = AdaBoostClassifier(n_estimators=200, learning_rate=0.05, random_state=0)
model.fit(X, y)

AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=0.05,
                   n_estimators=200, random_state=0)

In [0]:
preds = model.predict(test_data)

In [0]:
output = pd.DataFrame({'PassengerId':test_data.index+892, 'Survived':preds})
output.to_csv('./submission.csv', index=False)

In [0]:
from google.colab import files

files.download('./submission.csv')